In [1]:
import random
import os
import os.path as osp
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
import seaborn as sns
import pickle
import torch
import glob
from scipy.stats import entropy
from collections import defaultdict, Counter

pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

In [2]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def ratio_double_compressed(mean_difference, final_QP):
    # mean_difference = mean_difference[0]
    # final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))

        
    # エネルギー比を計算して閾値と比較
    if energy > 0:
        return right_energy / energy
    
    else:
        return 0

    
def is_double_compressed(mean_difference, final_QP, threshold):
    mean_difference = mean_difference[0]
    final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))
    # right_energy = np.sum(np.square(mean_difference[final_QP+1:52]))
    
    # print('energy: ', energy)
    # print('R-energy: ', right_energy)
    # print('Ratio: ', right_energy / energy)
    
    
    # エネルギー比を計算して閾値と比較
    if energy <= 0:
        return -1
    
    elif (right_energy / energy) != 0 and (right_energy / energy) > threshold:
        return True
    
    elif (right_energy / energy) != 0 and (right_energy / energy) <= threshold:
        return False
    
    else:
        return -1

def calculate_mae(file_path):
    try:
        with open(file_path, 'rb') as file:
            loaded_data, loaded_data_shifted = pickle.load(file)
    except Exception as e:
        print(f"Error occurred while loading {file_path}: {e}")
        return None

    # タプル内のリストを抽出
    original_mae = np.array(loaded_data)
    shifted_mae = np.array(loaded_data_shifted)

    # Coding ghostを計算してリストに格納する
    mae_difference = shifted_mae - original_mae
    
    # mae_differenceの各要素においてマイナスの値を0に変換
    # mae_difference_positive = np.maximum(mae_difference, 0)
    
    return mae_difference

In [3]:
rootpath_csv = "/Prove/Yoshihisa/HEIF_ghost/HEIF_IMAGES_CSV/"
rootpath_pkl = "/Prove/Yoshihisa/HEIF_ghost/PKL/"

train_list1 = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]
train_list2 = ["31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59", "60"]
train_list3 = ["61", "62", "63", "64", "65", "66", "67", "68", "69", "70", "71", "72", "73", "74", "75", "76", "77", "78", "79", "80", "81", "82", "83", "84", "85", "86", "87", "88", "89", "90"]
train_list4 = ["91", "92", "93", "94", "95", "96", "97", "98", "99", "100", "101", "102", "103", "104", "105", "106", "107", "108", "109", "110", "111", "112", "113", "114", "115", "116", "117", "118", "119", "120"]
train_list5 = ["121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "132", "133", "134", "135", "136", "137", "138", "139", "140", "141", "142", "143", "144", "145", "146", "147", "148", "149", "150"]
train_list6 = ["151", "152", "153", "154", "155", "156", "157", "158", "159", "160", "161", "162", "163", "164", "165", "166", "167", "168", "169", "170", "171", "172", "173", "174", "175", "176", "177", "178", "179", "180"]
train_list7 = ["181", "182", "183", "184", "185", "186", "187", "188", "189", "190", "191", "192", "193", "194", "195", "196", "197", "198", "199", "200", "201", "202", "203", "204", "205", "206", "207", "208", "209", "210"]
train_list8 = ["211", "212", "213", "214", "215", "216", "217", "218", "219", "220", "221", "222", "223", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "236", "237", "238", "239", "240"]
train_list9 = ["241", "242", "243", "244", "245", "246", "247", "248", "249", "250", "251", "252", "253", "254", "255", "256", "257", "258", "259", "260", "261", "262", "263", "264", "265", "266", "267", "268", "269", "270"]
train_list10 = ["271", "272", "273", "274", "275", "276", "277", "278", "279", "280", "281", "282", "283", "284", "285", "286", "287", "288", "289", "290", "291", "292", "293", "294", "295", "296", "297", "298", "299", "300"]

all_train_lists = [train_list1, train_list2, train_list3, train_list4, train_list5,
                   train_list6, train_list7, train_list8, train_list9, train_list10]

# すべてのリストを1つのリストに結合する
combined_train_list = sum(all_train_lists, [])

# リストの順序をランダムにシャッフルする
random.shuffle(combined_train_list)

# シャッフルされたリストを10個のグループに分割する
train_lists = [combined_train_list[i:i+30] for i in range(0, len(combined_train_list), 30)]
print(train_lists)



# CSV関連のリストを生成
csv_single_listsA = [[] for _ in range(10)]
csv_single_recompress_listsA = [[] for _ in range(10)]
csv_second_largeQP1_listsA = [[] for _ in range(10)]
csv_second_recompress_largeQP1_listsA = [[] for _ in range(10)]
csv_second_sameQP_listsA = [[] for _ in range(10)]
csv_second_recompress_sameQP_listsA = [[] for _ in range(10)]
csv_second_largeQP2_listsA = [[] for _ in range(10)]
csv_second_recompress_largeQP2_listsA = [[] for _ in range(10)]

def process_csv_lists(rootpath, train_list, single_list, single_recompress_list, 
                      second_largeQP1_list, second_recompress_largeQP1_list, 
                      second_sameQP_list, second_recompress_sameQP_list,
                      second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath, f'HEIF_images_single_csv/{image}_*')
        single_recompress_path = osp.join(rootpath, f'HEIF_images_second_sameQP_csv/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath, f'HEIF_images_second_csv/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath, f'HEIF_images_triple_csv/{image}_*')
        
        second_sameQP_path = osp.join(rootpath, f'HEIF_images_second_sameQP_csv/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath, f'HEIF_images_triple_sameQP_csv/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath, f'HEIF_images_second_largeQP_csv/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath, f'HEIF_images_triple_largeQP_csv/{image}_*')
        
        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)

# 各カテゴリのCSVリストを生成
for train_list, single_list, single_recompress_list in zip(train_lists, 
                                                           csv_single_listsA,
                                                           csv_single_recompress_listsA):
    process_csv_lists(rootpath_csv, train_list, single_list, single_recompress_list, 
                      [], [], [], [], [], [])


for train_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                   csv_second_largeQP1_listsA,
                                                                                                                                                                                                                   csv_second_recompress_largeQP1_listsA,
                                                                                                                                                                                                                   csv_second_sameQP_listsA,
                                                                                                                                                                                                                   csv_second_recompress_sameQP_listsA,
                                                                                                                                                                                                                   csv_second_largeQP2_listsA,
                                                                                                                                                                                                                   csv_second_recompress_largeQP2_listsA):
    process_csv_lists(rootpath_csv, train_list, [], [], 
                      second_largeQP1_list, second_recompress_largeQP1_list, 
                      second_sameQP_list, second_recompress_sameQP_list,
                      second_largeQP2_list, second_recompress_largeQP2_list)

    

    
# 出力リストを初期化
pkl_single_listsA = [[] for _ in range(10)]
pkl_single_recompress_listsA = [[] for _ in range(10)]
pkl_second_largeQP1_listsA = [[] for _ in range(10)]
pkl_second_recompress_largeQP1_listsA = [[] for _ in range(10)]
pkl_second_sameQP_listsA = [[] for _ in range(10)]
pkl_second_recompress_sameQP_listsA = [[] for _ in range(10)]
pkl_second_largeQP2_listsA = [[] for _ in range(10)]
pkl_second_recompress_largeQP2_listsA = [[] for _ in range(10)]    

def process_train_lists_pkl(rootpath, train_list, single_list, single_recompress_list, 
                            second_largeQP1_list, second_recompress_largeQP1_list, 
                            second_sameQP_list, second_recompress_sameQP_list,
                            second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath, f'pkl_single/{image}_*')
        single_recompress_path = osp.join(rootpath, f'pkl_second_sameQP/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath, f'pkl_second/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath, f'pkl_triple/{image}_*')
        
        second_sameQP_path = osp.join(rootpath, f'pkl_second_sameQP/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath, f'pkl_triple_sameQP/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath, f'pkl_second_largeQP/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath, f'pkl_triple_largeQP/{image}_*')
        

        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
                
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)

# 各カテゴリのリストを生成
for train_list, single_list, single_recompress_list in zip(train_lists, 
                                                           pkl_single_listsA,
                                                           pkl_single_recompress_listsA):
    process_train_lists_pkl(rootpath_pkl, train_list, single_list, single_recompress_list, 
                            [], [], [], [], [], [])


for train_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                   pkl_second_largeQP1_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_largeQP1_listsA,
                                                                                                                                                                                                                   pkl_second_sameQP_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_sameQP_listsA,
                                                                                                                                                                                                                   pkl_second_largeQP2_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_largeQP2_listsA):
    process_train_lists_pkl(rootpath_pkl, train_list, [], [], 
                            second_largeQP1_list, second_recompress_largeQP1_list, 
                            second_sameQP_list, second_recompress_sameQP_list,
                            second_largeQP2_list, second_recompress_largeQP2_list)


print("\nCSV Single ListsA:")
for i, lst in enumerate(csv_single_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Single Recompress ListsA:")
for i, lst in enumerate(csv_single_recompress_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Large QP1 ListsA:")
for i, lst in enumerate(csv_second_largeQP1_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Large QP1 ListsA:")
for i, lst in enumerate(csv_second_recompress_largeQP1_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Same QP ListsA:")
for i, lst in enumerate(csv_second_sameQP_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Same QP ListsA:")
for i, lst in enumerate(csv_second_recompress_sameQP_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Large QP2 ListsA:")
for i, lst in enumerate(csv_second_largeQP2_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Large QP2 ListsA:")
for i, lst in enumerate(csv_second_recompress_largeQP2_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

# 出力リストを表示
print("\nPKL Single ListsA:")
for i, lst in enumerate(pkl_single_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Single Recompress ListsA:")
for i, lst in enumerate(pkl_single_recompress_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Large QP1 ListsA:")
for i, lst in enumerate(pkl_second_largeQP1_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Large QP1 ListsA:")
for i, lst in enumerate(pkl_second_recompress_largeQP1_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Same QP ListsA:")
for i, lst in enumerate(pkl_second_sameQP_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Same QP ListsA:")
for i, lst in enumerate(pkl_second_recompress_sameQP_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Large QP2 ListsA:")
for i, lst in enumerate(pkl_second_largeQP2_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Large QP2 ListsA:")
for i, lst in enumerate(pkl_second_recompress_largeQP2_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

[['56', '25', '246', '133', '87', '139', '261', '216', '260', '269', '73', '148', '151', '255', '145', '38', '226', '222', '131', '18', '43', '164', '267', '147', '256', '233', '172', '100', '296', '195'], ['116', '218', '24', '115', '46', '208', '277', '127', '6', '297', '173', '245', '130', '109', '136', '126', '205', '92', '26', '215', '44', '15', '240', '278', '112', '68', '22', '157', '249', '42'], ['21', '273', '3', '290', '242', '14', '206', '75', '118', '166', '53', '236', '231', '280', '132', '93', '239', '122', '78', '81', '66', '142', '58', '86', '170', '251', '137', '281', '141', '289'], ['209', '287', '241', '254', '171', '94', '161', '168', '200', '224', '41', '135', '17', '10', '176', '293', '34', '158', '134', '74', '108', '188', '295', '227', '220', '102', '121', '63', '182', '60'], ['30', '268', '223', '193', '85', '83', '52', '234', '271', '235', '32', '229', '283', '27', '11', '184', '286', '123', '210', '54', '264', '180', '152', '217', '105', '238', '12', '88', '2

In [4]:
# single_listsおよびsingle_recompress_listsは初期化されている前提
single_csv1 = list(zip(csv_single_listsA[0], pkl_single_listsA[0], csv_single_recompress_listsA[0], pkl_single_recompress_listsA[0]))
single_csv2 = list(zip(csv_single_listsA[1], pkl_single_listsA[1], csv_single_recompress_listsA[1], pkl_single_recompress_listsA[1]))
single_csv3 = list(zip(csv_single_listsA[2], pkl_single_listsA[2], csv_single_recompress_listsA[2], pkl_single_recompress_listsA[2]))
single_csv4 = list(zip(csv_single_listsA[3], pkl_single_listsA[3], csv_single_recompress_listsA[3], pkl_single_recompress_listsA[3]))
single_csv5 = list(zip(csv_single_listsA[4], pkl_single_listsA[4], csv_single_recompress_listsA[4], pkl_single_recompress_listsA[4]))
single_csv6 = list(zip(csv_single_listsA[5], pkl_single_listsA[5], csv_single_recompress_listsA[5], pkl_single_recompress_listsA[5]))
single_csv7 = list(zip(csv_single_listsA[6], pkl_single_listsA[6], csv_single_recompress_listsA[6], pkl_single_recompress_listsA[6]))
single_csv8 = list(zip(csv_single_listsA[7], pkl_single_listsA[7], csv_single_recompress_listsA[7], pkl_single_recompress_listsA[7]))
single_csv9 = list(zip(csv_single_listsA[8], pkl_single_listsA[8], csv_single_recompress_listsA[8], pkl_single_recompress_listsA[8]))
single_csv10 = list(zip(csv_single_listsA[9], pkl_single_listsA[9], csv_single_recompress_listsA[9], pkl_single_recompress_listsA[9]))
print(len(single_csv10))

300


In [5]:
# Large_QP1
second_largeQP1_csv1 = list(zip(csv_second_largeQP1_listsA[0], pkl_second_largeQP1_listsA[0], csv_second_recompress_largeQP1_listsA[0], pkl_second_recompress_largeQP1_listsA[0]))
second_largeQP1_csv2 = list(zip(csv_second_largeQP1_listsA[1], pkl_second_largeQP1_listsA[1], csv_second_recompress_largeQP1_listsA[1], pkl_second_recompress_largeQP1_listsA[1]))
second_largeQP1_csv3 = list(zip(csv_second_largeQP1_listsA[2], pkl_second_largeQP1_listsA[2], csv_second_recompress_largeQP1_listsA[2], pkl_second_recompress_largeQP1_listsA[2]))
second_largeQP1_csv4 = list(zip(csv_second_largeQP1_listsA[3], pkl_second_largeQP1_listsA[3], csv_second_recompress_largeQP1_listsA[3], pkl_second_recompress_largeQP1_listsA[3]))
second_largeQP1_csv5 = list(zip(csv_second_largeQP1_listsA[4], pkl_second_largeQP1_listsA[4], csv_second_recompress_largeQP1_listsA[4], pkl_second_recompress_largeQP1_listsA[4]))
second_largeQP1_csv6 = list(zip(csv_second_largeQP1_listsA[5], pkl_second_largeQP1_listsA[5], csv_second_recompress_largeQP1_listsA[5], pkl_second_recompress_largeQP1_listsA[5]))
second_largeQP1_csv7 = list(zip(csv_second_largeQP1_listsA[6], pkl_second_largeQP1_listsA[6], csv_second_recompress_largeQP1_listsA[6], pkl_second_recompress_largeQP1_listsA[6]))
second_largeQP1_csv8 = list(zip(csv_second_largeQP1_listsA[7], pkl_second_largeQP1_listsA[7], csv_second_recompress_largeQP1_listsA[7], pkl_second_recompress_largeQP1_listsA[7]))
second_largeQP1_csv9 = list(zip(csv_second_largeQP1_listsA[8], pkl_second_largeQP1_listsA[8], csv_second_recompress_largeQP1_listsA[8], pkl_second_recompress_largeQP1_listsA[8]))
second_largeQP1_csv10 = list(zip(csv_second_largeQP1_listsA[9], pkl_second_largeQP1_listsA[9], csv_second_recompress_largeQP1_listsA[9], pkl_second_recompress_largeQP1_listsA[9]))
print(len(second_largeQP1_csv1))


second_largeQP1_csv1 = random.sample(second_largeQP1_csv1, 100)
second_largeQP1_csv2 = random.sample(second_largeQP1_csv2, 100)
second_largeQP1_csv3 = random.sample(second_largeQP1_csv3, 100)
second_largeQP1_csv4 = random.sample(second_largeQP1_csv4, 100)
second_largeQP1_csv5 = random.sample(second_largeQP1_csv5, 100)
second_largeQP1_csv6 = random.sample(second_largeQP1_csv6, 100)
second_largeQP1_csv7 = random.sample(second_largeQP1_csv7, 100)
second_largeQP1_csv8 = random.sample(second_largeQP1_csv8, 100)
second_largeQP1_csv9 = random.sample(second_largeQP1_csv9, 100)
# second_largeQP1_csv10 = selected_data[9]
print('\ndouble images train by QP1>QP2: ', len(second_largeQP1_csv1))


second_largeQP1_csv10 = random.sample(second_largeQP1_csv10, 300)
print('\ndouble images test by QP1>QP2: ', len(second_largeQP1_csv10))

1710

double images train by QP1>QP2:  100

double images test by QP1>QP2:  300


In [6]:
# sameQP
second_sameQP_csv1 = list(zip(csv_second_sameQP_listsA[0], pkl_second_sameQP_listsA[0], csv_second_recompress_sameQP_listsA[0], pkl_second_recompress_sameQP_listsA[0]))
second_sameQP_csv2 = list(zip(csv_second_sameQP_listsA[1], pkl_second_sameQP_listsA[1], csv_second_recompress_sameQP_listsA[1], pkl_second_recompress_sameQP_listsA[1]))
second_sameQP_csv3 = list(zip(csv_second_sameQP_listsA[2], pkl_second_sameQP_listsA[2], csv_second_recompress_sameQP_listsA[2], pkl_second_recompress_sameQP_listsA[2]))
second_sameQP_csv4 = list(zip(csv_second_sameQP_listsA[3], pkl_second_sameQP_listsA[3], csv_second_recompress_sameQP_listsA[3], pkl_second_recompress_sameQP_listsA[3]))
second_sameQP_csv5 = list(zip(csv_second_sameQP_listsA[4], pkl_second_sameQP_listsA[4], csv_second_recompress_sameQP_listsA[4], pkl_second_recompress_sameQP_listsA[4]))
second_sameQP_csv6 = list(zip(csv_second_sameQP_listsA[5], pkl_second_sameQP_listsA[5], csv_second_recompress_sameQP_listsA[5], pkl_second_recompress_sameQP_listsA[5]))
second_sameQP_csv7 = list(zip(csv_second_sameQP_listsA[6], pkl_second_sameQP_listsA[6], csv_second_recompress_sameQP_listsA[6], pkl_second_recompress_sameQP_listsA[6]))
second_sameQP_csv8 = list(zip(csv_second_sameQP_listsA[7], pkl_second_sameQP_listsA[7], csv_second_recompress_sameQP_listsA[7], pkl_second_recompress_sameQP_listsA[7]))
second_sameQP_csv9 = list(zip(csv_second_sameQP_listsA[8], pkl_second_sameQP_listsA[8], csv_second_recompress_sameQP_listsA[8], pkl_second_recompress_sameQP_listsA[8]))
second_sameQP_csv10 = list(zip(csv_second_sameQP_listsA[9], pkl_second_sameQP_listsA[9], csv_second_recompress_sameQP_listsA[9], pkl_second_recompress_sameQP_listsA[9]))
print(len(second_sameQP_csv10))

second_sameQP_csv1 = random.sample(second_sameQP_csv1, 100)
second_sameQP_csv2 = random.sample(second_sameQP_csv2, 100)
second_sameQP_csv3 = random.sample(second_sameQP_csv3, 100)
second_sameQP_csv4 = random.sample(second_sameQP_csv4, 100)
second_sameQP_csv5 = random.sample(second_sameQP_csv5, 100)
second_sameQP_csv6 = random.sample(second_sameQP_csv6, 100)
second_sameQP_csv7 = random.sample(second_sameQP_csv7, 100)
second_sameQP_csv8 = random.sample(second_sameQP_csv8, 100)
second_sameQP_csv9 = random.sample(second_sameQP_csv9, 100)
print('\ndouble images train by QP1=QP2: ',len(second_sameQP_csv9))

second_sameQP_csv10 = random.sample(second_sameQP_csv10, 300)
print('\ndouble images test by QP1=QP2: ',len(second_sameQP_csv10))

300

double images train by QP1=QP2:  100

double images test by QP1=QP2:  300


In [7]:
# Large_QP2
second_largeQP2_csv1 = list(zip(csv_second_largeQP2_listsA[0], pkl_second_largeQP2_listsA[0], csv_second_recompress_largeQP2_listsA[0], pkl_second_recompress_largeQP2_listsA[0]))
second_largeQP2_csv2 = list(zip(csv_second_largeQP2_listsA[1], pkl_second_largeQP2_listsA[1], csv_second_recompress_largeQP2_listsA[1], pkl_second_recompress_largeQP2_listsA[1]))
second_largeQP2_csv3 = list(zip(csv_second_largeQP2_listsA[2], pkl_second_largeQP2_listsA[2], csv_second_recompress_largeQP2_listsA[2], pkl_second_recompress_largeQP2_listsA[2]))
second_largeQP2_csv4 = list(zip(csv_second_largeQP2_listsA[3], pkl_second_largeQP2_listsA[3], csv_second_recompress_largeQP2_listsA[3], pkl_second_recompress_largeQP2_listsA[3]))
second_largeQP2_csv5 = list(zip(csv_second_largeQP2_listsA[4], pkl_second_largeQP2_listsA[4], csv_second_recompress_largeQP2_listsA[4], pkl_second_recompress_largeQP2_listsA[4]))
second_largeQP2_csv6 = list(zip(csv_second_largeQP2_listsA[5], pkl_second_largeQP2_listsA[5], csv_second_recompress_largeQP2_listsA[5], pkl_second_recompress_largeQP2_listsA[5]))
second_largeQP2_csv7 = list(zip(csv_second_largeQP2_listsA[6], pkl_second_largeQP2_listsA[6], csv_second_recompress_largeQP2_listsA[6], pkl_second_recompress_largeQP2_listsA[6]))
second_largeQP2_csv8 = list(zip(csv_second_largeQP2_listsA[7], pkl_second_largeQP2_listsA[7], csv_second_recompress_largeQP2_listsA[7], pkl_second_recompress_largeQP2_listsA[7]))
second_largeQP2_csv9 = list(zip(csv_second_largeQP2_listsA[8], pkl_second_largeQP2_listsA[8], csv_second_recompress_largeQP2_listsA[8], pkl_second_recompress_largeQP2_listsA[8]))
second_largeQP2_csv10 = list(zip(csv_second_largeQP2_listsA[9], pkl_second_largeQP2_listsA[9], csv_second_recompress_largeQP2_listsA[9], pkl_second_recompress_largeQP2_listsA[9]))
print(len(second_largeQP2_csv1))

second_largeQP2_csv1 = random.sample(second_largeQP2_csv1, 100)
second_largeQP2_csv2 = random.sample(second_largeQP2_csv2, 100)
second_largeQP2_csv3 = random.sample(second_largeQP2_csv3, 100)
second_largeQP2_csv4 = random.sample(second_largeQP2_csv4, 100)
second_largeQP2_csv5 = random.sample(second_largeQP2_csv5, 100)
second_largeQP2_csv6 = random.sample(second_largeQP2_csv6, 100)
second_largeQP2_csv7 = random.sample(second_largeQP2_csv7, 100)
second_largeQP2_csv8 = random.sample(second_largeQP2_csv8, 100)
second_largeQP2_csv9 = random.sample(second_largeQP2_csv9, 100)
# second_largeQP2_csv10 = selected_data[9]
print('\ndouble images train by QP1<QP2: ', len(second_largeQP2_csv1))

second_largeQP2_csv10 = random.sample(second_largeQP2_csv10, 300)
print('\ndouble images test by QP1<QP2: ', len(second_largeQP2_csv10))

1170

double images train by QP1<QP2:  100

double images test by QP1<QP2:  300


In [8]:
train_csv_list1 = single_csv1 + second_largeQP1_csv1 + second_sameQP_csv1 + second_largeQP2_csv1
train_csv_list2 = single_csv2 + second_largeQP1_csv2 + second_sameQP_csv2 + second_largeQP2_csv2
train_csv_list3 = single_csv3 + second_largeQP1_csv3 + second_sameQP_csv3 + second_largeQP2_csv3
train_csv_list4 = single_csv4 + second_largeQP1_csv4 + second_sameQP_csv4 + second_largeQP2_csv4
train_csv_list5 = single_csv5 + second_largeQP1_csv5 + second_sameQP_csv5 + second_largeQP2_csv5
train_csv_list6 = single_csv6 + second_largeQP1_csv6 + second_sameQP_csv6 + second_largeQP2_csv6
train_csv_list7 = single_csv7 + second_largeQP1_csv7 + second_sameQP_csv7 + second_largeQP2_csv7
train_csv_list8 = single_csv8 + second_largeQP1_csv8 + second_sameQP_csv8 + second_largeQP2_csv8
train_csv_list9 = single_csv9 + second_largeQP1_csv9 + second_sameQP_csv9 + second_largeQP2_csv9
print("train_csv_list: ", len(train_csv_list9))

test_csv_largeQP1 = single_csv10 + second_largeQP1_csv10
test_csv_sameQP = single_csv10 + second_sameQP_csv10
test_csv_largeQP2 = single_csv10 + second_largeQP2_csv10

print("\ntest_csv_largeQP1", len(test_csv_largeQP1))
print("test_csv_sameQP", len(test_csv_sameQP))
print("test_csv_largeQP2", len(test_csv_largeQP2))

train_csv_list:  600

test_csv_largeQP1 600
test_csv_sameQP 600
test_csv_largeQP2 600


In [9]:
def laplace_smoothing(probabilities, alpha=1):
    """
    ラプラス平滑化を行う関数
    
    Args:
    probabilities (list): 平滑化する確率分布のリスト
    alpha (float): 平滑化パラメータ
    
    Returns:
    smoothed_probabilities (list): 平滑化された確率分布のリスト
    """
    total_count = sum(probabilities)
    num_elements = len(probabilities)
    
    smoothed_probabilities = [(count + alpha) / (total_count + alpha * num_elements) for count in probabilities]
    
    return smoothed_probabilities


def process_train_csv_lists(train_csv_list):
    pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
                  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

#     luminance_columns = ["LU1_0","LU1_1","LU1_2","LU1_3",
#                          "LU1_4","LU1_5","LU1_6","LU1_7",
#                          "LU1_8","LU1_9","LU1_10","LU1_11",
#                          "LU1_12","LU1_13","LU1_14","LU1_15",
#                          "LU1_16","LU1_17","LU1_18","LU1_19",
#                          "LU1_20","LU1_21","LU1_22","LU1_23",
#                          "LU1_24","LU1_25","LU1_26","LU1_27",
#                          "LU1_28","LU1_29","LU1_30","LU1_31",
#                          "LU1_32","LU1_33","LU1_34",
                         
#                          "LU2_0","LU2_1","LU2_2","LU2_3",
#                          "LU2_4","LU2_5","LU2_6","LU2_7",
#                          "LU2_8","LU2_9","LU2_10","LU2_11",
#                          "LU2_12","LU2_13","LU2_14","LU2_15",
#                          "LU2_16","LU2_17","LU2_18","LU2_19",
#                          "LU2_20","LU2_21","LU2_22","LU2_23",
#                          "LU2_24","LU2_25","LU2_26","LU2_27",
#                          "LU2_28","LU2_29","LU2_30","LU2_31",
#                          "LU2_32","LU2_33","LU2_34"]
    
    luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27",
                         "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11", "LU2_25","LU2_26","LU2_27"]

    chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                           "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]
    
    
    
    label_columns = ["LABEL"]
    mae1_columns = [f"MAE1_{i}" for i in range(52)]
    mae2_columns = [f"MAE2_{i}" for i in range(52)]
    mae_columns = ["MAE"]
    final_qp_columns = ["FINAL_QP"]
    kl_divergence1 = ["KLD_PU"]
    kl_divergence2 = ["KLD_LUMA"]
    kl_divergence3 = ["KLD_CHROMA"]
    ratio_columns1 = ["RATIO1"]
    ratio_columns2 = ["RATIO2"]
    
    train_df1_1 = pd.DataFrame(columns=pu_columns)
    train_df1_2 = pd.DataFrame(columns=luminance_columns)
    train_df1_3 = pd.DataFrame(columns=chrominance_columns)
    LABEL = pd.DataFrame(columns=label_columns)
    RATIO1 = pd.DataFrame(columns=ratio_columns1)
    RATIO2 = pd.DataFrame(columns=ratio_columns2)
    train_df3 = pd.DataFrame(columns=mae1_columns)
    train_df4 = pd.DataFrame(columns=mae2_columns)
    MAE = pd.DataFrame(columns=mae_columns)
    FINAL_QP = pd.DataFrame(columns=final_qp_columns)
    kl_divergence_df1 = pd.DataFrame(columns=kl_divergence1)
    kl_divergence_df2 = pd.DataFrame(columns=kl_divergence2)
    kl_divergence_df3 = pd.DataFrame(columns=kl_divergence3)

    for path1, path2, path3, path4 in train_csv_list:
        label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
        train_pkl_list = [path2, path4]
        df1 = pd.read_csv(path1)
        df2 = pd.read_csv(path3)
        
        # 平滑化を行う
        probabilities_df1 = laplace_smoothing([df1.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        probabilities_df2 = laplace_smoothing([df2.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        kl_divergence1 = entropy(probabilities_df1, probabilities_df2)
        
        probabilities_df3 = laplace_smoothing([df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        probabilities_df4 = laplace_smoothing([df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        kl_divergence2 = entropy(probabilities_df3, probabilities_df4)
        
        probabilities_df5 = laplace_smoothing([df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        probabilities_df6 = laplace_smoothing([df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        kl_divergence3 = entropy(probabilities_df5, probabilities_df6)
        
        
        pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
        # lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
        lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
        ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
        
        train_df1_1 = pd.concat([train_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
        train_df1_2= pd.concat([train_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
        train_df1_3 = pd.concat([train_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
        kl_divergence_df1 = pd.concat([kl_divergence_df1, pd.DataFrame({"KLD_PU": [kl_divergence1]})], ignore_index=True)
        kl_divergence_df2 = pd.concat([kl_divergence_df2, pd.DataFrame({"KLD_LUMA": [kl_divergence2]})], ignore_index=True)
        kl_divergence_df3 = pd.concat([kl_divergence_df3, pd.DataFrame({"KLD_CHROMA": [kl_divergence3]})], ignore_index=True)


        LABEL = pd.concat([LABEL, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

        final_QP = extract_finalQP(train_pkl_list[0])

        mae_d1 = calculate_mae(train_pkl_list[0])
        mae_d2 = calculate_mae(train_pkl_list[1])
        ratio1 = ratio_double_compressed(mae_d1, final_QP)
        ratio2 = ratio_double_compressed(mae_d2, final_QP)

        RATIO1 = pd.concat([RATIO1, pd.DataFrame({"RATIO1": [ratio1]})], ignore_index=True)
        RATIO2 = pd.concat([RATIO2, pd.DataFrame({"RATIO2": [ratio2]})], ignore_index=True)

        train_df3 = pd.concat([train_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
        train_df4 = pd.concat([train_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
        MAE = pd.concat([MAE, pd.DataFrame({"MAE": [mae_d1]})], ignore_index=True)
        FINAL_QP = pd.concat([FINAL_QP, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)

    train_df1_1.reset_index(drop=True, inplace=True)
    train_df1_2.reset_index(drop=True, inplace=True)
    train_df1_3.reset_index(drop=True, inplace=True)
    LABEL.reset_index(drop=True, inplace=True)
    RATIO1.reset_index(drop=True, inplace=True)
    RATIO2.reset_index(drop=True, inplace=True)
    kl_divergence_df1.reset_index(drop=True, inplace=True)
    kl_divergence_df2.reset_index(drop=True, inplace=True)
    kl_divergence_df3.reset_index(drop=True, inplace=True)

    # train_df = pd.concat([train_df1_1, train_df1_2, train_df1_3, train_df3, train_df4], axis=1)
    train_df = pd.concat([FINAL_QP, train_df1_1, train_df1_2, train_df1_3, kl_divergence_df1, kl_divergence_df2, kl_divergence_df3, RATIO1, RATIO2], axis=1)
    train_df_onlyGhost = pd.concat([FINAL_QP, kl_divergence_df1, kl_divergence_df2, kl_divergence_df3, RATIO1, RATIO2], axis=1)

    return train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP


In [10]:
train_df1, train_df_onlyGhost1, LABEL1, MAE1, FINAL_QP1 = process_train_csv_lists(train_csv_list1)
train_df2, train_df_onlyGhost2, LABEL2, MAE2, FINAL_QP2 = process_train_csv_lists(train_csv_list2)
train_df3, train_df_onlyGhost3, LABEL3, MAE3, FINAL_QP3 = process_train_csv_lists(train_csv_list3)
train_df4, train_df_onlyGhost4, LABEL4, MAE4, FINAL_QP4 = process_train_csv_lists(train_csv_list4)
train_df5, train_df_onlyGhost5, LABEL5, MAE5, FINAL_QP5 = process_train_csv_lists(train_csv_list5)
train_df6, train_df_onlyGhost6, LABEL6, MAE6, FINAL_QP6 = process_train_csv_lists(train_csv_list6)
train_df7, train_df_onlyGhost7, LABEL7, MAE7, FINAL_QP7 = process_train_csv_lists(train_csv_list7)
train_df8, train_df_onlyGhost8, LABEL8, MAE8, FINAL_QP8 = process_train_csv_lists(train_csv_list8)
train_df9, train_df_onlyGhost9, LABEL9, MAE9, FINAL_QP9 = process_train_csv_lists(train_csv_list9)

test_df1, test_df_onlyGhost1, LABEL_t1, MAE_t1, FINAL_QP_t1 = process_train_csv_lists(test_csv_largeQP1)
test_df2, test_df_onlyGhost2, LABEL_t2, MAE_t2, FINAL_QP_t2 = process_train_csv_lists(test_csv_sameQP)
test_df3, test_df_onlyGhost3, LABEL_t3, MAE_t3, FINAL_QP_t3 = process_train_csv_lists(test_csv_largeQP2)

In [11]:
# 各データフレームを結合
combined_train_df = pd.concat([train_df1, train_df2, train_df3, train_df4, train_df5, train_df6, train_df7, train_df8, train_df9], ignore_index=True)
combined_train_df_onlyGhost = pd.concat([train_df_onlyGhost1, train_df_onlyGhost2, train_df_onlyGhost3, train_df_onlyGhost4, train_df_onlyGhost5, train_df_onlyGhost6, train_df_onlyGhost7, train_df_onlyGhost8, train_df_onlyGhost9], ignore_index=True)
combined_LABEL = pd.concat([LABEL1, LABEL2, LABEL3, LABEL4, LABEL5, LABEL6, LABEL7, LABEL8, LABEL9], ignore_index=True)
combined_MAE = pd.concat([MAE1, MAE2, MAE3, MAE4, MAE5, MAE6, MAE7, MAE8, MAE9], ignore_index=True)
combined_FINAL_QP = pd.concat([FINAL_QP1, FINAL_QP2, FINAL_QP3, FINAL_QP4, FINAL_QP5, FINAL_QP6, FINAL_QP7, FINAL_QP8, FINAL_QP9], ignore_index=True)

In [12]:
print(combined_train_df.shape)
print(combined_train_df_onlyGhost.shape)
print(combined_LABEL.shape)
print(combined_MAE.shape)
print(combined_FINAL_QP.shape)

print(test_df1.shape)
print(test_df_onlyGhost1.shape)
print(LABEL_t1.shape)
print(MAE_t1.shape)
print(FINAL_QP_t1.shape)

print(test_df2.shape)
print(test_df_onlyGhost2.shape)
print(LABEL_t2.shape)
print(MAE_t2.shape)
print(FINAL_QP_t2.shape)

print(test_df3.shape)
print(test_df_onlyGhost3.shape)
print(LABEL_t3.shape)
print(MAE_t3.shape)
print(FINAL_QP_t3.shape)

(5400, 44)
(5400, 6)
(5400, 1)
(5400, 1)
(5400, 1)
(600, 44)
(600, 6)
(600, 1)
(600, 1)
(600, 1)
(600, 44)
(600, 6)
(600, 1)
(600, 1)
(600, 1)
(600, 44)
(600, 6)
(600, 1)
(600, 1)
(600, 1)


In [13]:
print("Before scaling:")
print("Combined Train DF:")
print(combined_train_df.head())
print("Combined Train DF Only Ghost:")
print(combined_train_df_onlyGhost.head())

print("\nBefore scaling:")
print("Combined Test DF:")
print(test_df1.head())
print("Combined Test DF Only Ghost:")
print(test_df_onlyGhost1.head())

Before scaling:
Combined Train DF:
  FINAL_QP PU1_64 PU1_32 PU1_16  PU1_8  PU1_4 PU2_64 PU2_32 PU2_16  PU2_8  PU2_4 LU1_0 LU1_1 LU1_9 LU1_10 LU1_11 LU1_25 LU1_26 LU1_27  LU2_0 LU2_1 LU2_9 LU2_10 LU2_11 LU2_25 LU2_26 LU2_27  CH1_0  CH1_1 CH1_10 CH1_26 CH1_34 CH1_36  CH2_0  CH2_1 CH2_10 CH2_26 CH2_34 CH2_36    KLD_PU  KLD_LUMA KLD_CHROMA    RATIO1    RATIO2
0       10      0   2816   4016  15596  37572      0   2816   3968  14568  38648  7027  5523  1407   1943   1265   1293   2259   1179   6969  5867  1407   1916   1256   1392   2093   1292  14032  10048   6180   6388   2020  21332  14092  10332   6308   6732   1872  20664  0.000847   0.00109   0.000512   0.03525  0.026403
1       16      0   5632   6016  17944  30408      0   5568   6016  17256  31160  7799  6432  1075   1767   1221   1427   1959   1121   7695  6867  1128   1780   1290   1487   1928   1007  12896   7760   4788   4468   1448  28640  12488   8572   4744   4524   1312  28360  0.000384  0.000987   0.000918  0.017283  0.010

In [14]:
def process_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP, scaler_main=None, scaler_ghost=None, fit_scaler=True):
    if fit_scaler:
        scaler_main = MinMaxScaler()
        scaler_ghost = MinMaxScaler()
        X_train = scaler_main.fit_transform(train_df)
        X_train_onlyGhost = scaler_ghost.fit_transform(train_df_onlyGhost)
    else:
        X_train = scaler_main.transform(train_df)
        X_train_onlyGhost = scaler_ghost.transform(train_df_onlyGhost)

    MAE_array = MAE.values
    FINAL_QP_array = FINAL_QP.values
    Y_train = LABEL['LABEL'].astype(int).values

    return X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train, scaler_main, scaler_ghost

# 訓練データのスケーリング
X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train, scaler_main, scaler_ghost = process_results_to_lists(
    combined_train_df, combined_train_df_onlyGhost, combined_LABEL, combined_MAE, combined_FINAL_QP, fit_scaler=True
)

# スケーリング後のデータを表示（任意）
print("After scaling:")
print("X_train:")
print(pd.DataFrame(X_train).head())
print("X_train_onlyGhost:")
print(pd.DataFrame(X_train_onlyGhost).head())

# データを元に戻すための関数
def restore_data_to_original_order(data, original_lengths):
    restored_data = []
    start_index = 0
    for length in original_lengths:
        restored_data.append(data[start_index:start_index + length])
        start_index += length
    return restored_data

# 元のデータフレームの長さ
original_lengths = [len(train_df1), len(train_df2), len(train_df3), len(train_df4), len(train_df5), 
                    len(train_df6), len(train_df7), len(train_df8), len(train_df9)]

# データを元の順序に戻す
X_train_list = restore_data_to_original_order(X_train, original_lengths)
X_train_onlyGhost_list = restore_data_to_original_order(X_train_onlyGhost, original_lengths)
MAE_list = restore_data_to_original_order(MAE_array, original_lengths)
FINAL_QP_list = restore_data_to_original_order(FINAL_QP_array, original_lengths)
Y_train_list = restore_data_to_original_order(Y_train, original_lengths)

# テストデータのスケーリング関数
def append_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list, scaler_main=None, scaler_ghost=None, fit_scaler=True):
    X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train, _, _ = process_results_to_lists(
        train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP, scaler_main, scaler_ghost, fit_scaler)
    X_train_list.append(X_train)
    X_train_onlyGhost_list.append(X_train_onlyGhost)
    MAE_list.append(MAE_array)
    FINAL_QP_list.append(FINAL_QP_array)
    Y_train_list.append(Y_train)
    return X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list

# テストデータ用のリストの初期化
X_test_list1 = []
X_test_onlyGhost_list1 = []
MAE_list_t1 = []
FINAL_QP_list_t1 = []
Y_test_list1 = []

X_test_list2 = []
X_test_onlyGhost_list2 = []
MAE_list_t2 = []
FINAL_QP_list_t2 = []
Y_test_list2 = []

X_test_list3 = []
X_test_onlyGhost_list3 = []
MAE_list_t3 = []
FINAL_QP_list_t3 = []
Y_test_list3 = []

# テストデータの処理とスケーリング
X_test_list1, X_test_onlyGhost_list1, MAE_list_t1, FINAL_QP_list_t1, Y_test_list1 = append_results_to_lists(
    test_df1, test_df_onlyGhost1, LABEL_t1, MAE_t1, FINAL_QP_t1, X_test_list1, X_test_onlyGhost_list1, MAE_list_t1, FINAL_QP_list_t1, Y_test_list1, scaler_main, scaler_ghost, fit_scaler=False
)
X_test_list2, X_test_onlyGhost_list2, MAE_list_t2, FINAL_QP_list_t2, Y_test_list2 = append_results_to_lists(
    test_df2, test_df_onlyGhost2, LABEL_t2, MAE_t2, FINAL_QP_t2, X_test_list2, X_test_onlyGhost_list2, MAE_list_t2, FINAL_QP_list_t2, Y_test_list2, scaler_main, scaler_ghost, fit_scaler=False
)
X_test_list3, X_test_onlyGhost_list3, MAE_list_t3, FINAL_QP_list_t3, Y_test_list3 = append_results_to_lists(
    test_df3, test_df_onlyGhost3, LABEL_t3, MAE_t3, FINAL_QP_t3, X_test_list3, X_test_onlyGhost_list3, MAE_list_t3, FINAL_QP_list_t3, Y_test_list3, scaler_main, scaler_ghost, fit_scaler=False
)

# 確認用の出力
print("\nTest data after scaling:")
print("X_test_list1:")
print(pd.DataFrame(X_test_list1[0]).head())
print("X_test_onlyGhost_list1:")
print(pd.DataFrame(X_test_onlyGhost_list1[0]).head())

After scaling:
X_train:
      0    1         2         3         4         5    6         7         8         9         10        11        12        13        14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        33        34        35        36        37        38        39        40        41        42        43
0  0.125  0.0  0.047826  0.126512  0.540627  0.671264  0.0  0.047774  0.124937  0.497269  0.673451  0.132091  0.113719  0.107987  0.090557  0.103296  0.087655  0.073389  0.124171  0.139556  0.116381  0.107895  0.090831  0.104420  0.094366  0.062468  0.140833  0.391387  0.406407  0.322278  0.390369  0.184576  0.308908  0.386336  0.408379  0.319360  0.380167  0.165722  0.297471  0.047515  0.009901  0.007633  0.033186  0.024965
1  0.275  0.0  0.095652  0.189516  0.622019  0.543272  0.0  0.094463  0.189421  0.589022  0.542971  0.157464  0.144158

In [15]:
# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 2000, 3000, 4000, 5000]}
# C_values = {'C': [100]}
# kfold = StratifiedKFold(n_splits=10, shuffle=True)
kfold = KFold(n_splits=9, shuffle=True, random_state=42)


results = pd.DataFrame(columns=['C_RBF_LQP1','Score_RBF_LQP1', 'tnr_rbf_lqp1', 'tpr_rbf_lqp1',
                                'C_RBF_SQP','Score_RBF_SQP', 'tnr_rbf_sqp', 'tpr_rbf_sqp',
                                'C_RBF_LQP2','Score_RBF_LQP2', 'tnr_rbf_lqp2', 'tpr_rbf_lqp2',
                                'C_LINEAR_LQP1','Score_LINEAR_LQP1', 'tnr_linear_lqp1', 'tpr_linear_lqp1',
                                'C_LINEAR_SQP','Score_LINEAR_SQP', 'tnr_linear_sqp', 'tpr_linear_sqp',
                                'C_LINEAR_LQP2','Score_LINEAR_LQP2', 'tnr_linear_lqp2', 'tpr_linear_lqp2',
                                
                                'C_RBF2_LQP1','Score_RBF2_LQP1', 'tnr_rbf2_lqp1', 'tpr_rbf2_lqp1',
                                'C_RBF2_SQP','Score_RBF2_SQP', 'tnr_rbf2_sqp', 'tpr_rbf2_sqp',
                                'C_RBF2_LQP2','Score_RBF2_LQP2', 'tnr_rbf2_lqp2', 'tpr_rbf2_lqp2',
                                'C_LINEAR2_LQP1','Score_LINEAR2_LQP1', 'tnr_linear2_lqp1', 'tpr_linear2_lqp1',
                                'C_LINEAR2_SQP','Score_LINEAR2_SQP', 'tnr_linear2_sqp2', 'tpr_linear2_sqp',
                                'C_LINEAR2_LQP2','Score_LINEAR2_LQP2', 'tnr_linear2_lqp2', 'tpr_linear2_lqp2',
                                
                                'Threshold_LQP1', 'LQP1_old', 'tnr_old_lqp1', 'tpr_old_lqp1',
                                'Threshold_SQP', 'SQP_old', 'tnr_old_sqp', 'tpr_old_sqp',
                                'Threshold_LQP2', 'LQP2_old', 'tnr_old_lqp2', 'tpr_old_lqp2'])


# results = pd.DataFrame(columns=['C_RBF_LQP1','Score_RBF_LQP1', 'tnr_rbf_lqp1', 'tpr_rbf_lqp1',
#                                 'C_RBF_SQP','Score_RBF_SQP', 'tnr_rbf_sqp', 'tpr_rbf_sqp',
#                                 'C_RBF_LQP2','Score_RBF_LQP2', 'tnr_rbf_lqp2', 'tpr_rbf_lqp2',
#                                 'C_LINEAR_LQP1','Score_LINEAR_LQP1', 'tnr_linear_lqp1', 'tpr_linear_lqp1',
#                                 'C_LINEAR_SQP','Score_LINEAR_SQP', 'tnr_linear_sqp', 'tpr_linear_sqp',
#                                 'C_LINEAR_LQP2','Score_LINEAR_LQP2', 'tnr_linear_lqp2', 'tpr_linear_lqp2',
                                                        
#                                 'Threshold_LQP1', 'LQP1_old', 'tnr_old_lqp1', 'tpr_old_lqp1',
#                                 'Threshold_SQP', 'SQP_old', 'tnr_old_sqp', 'tpr_old_sqp',
#                                 'Threshold_LQP2', 'LQP2_old', 'tnr_old_lqp2', 'tpr_old_lqp2'])

    
X_index = np.arange(9)  # インデックスとして0から8までの数字を用意

# ループで各分割のtrain_idsとtest_idsを取得
for fold, (train_ids, test_ids) in enumerate(kfold.split(X_index)):
    print(f"<Fold-{fold+1}>")
    print("Train indices:", train_ids)
    print("Test indices:", test_ids)
    
    train_data = [X_train_list[i] for i in train_ids]
    train_data_OG = [X_train_onlyGhost_list[i] for i in train_ids]
    train_label = [Y_train_list[i] for i in train_ids]
    
    val_data = [X_train_list[i] for i in test_ids]
    val_data_OG = [X_train_onlyGhost_list[i] for i in test_ids]
    val_label = [Y_train_list[i] for i in test_ids]
    
    X_train = [item for data in train_data for item in data]
    X_train_OG = [item for data in train_data_OG for item in data]
    Y_train = [item for data in train_label for item in data]
    
    X_val = [item for data in val_data for item in data]
    X_val_OG = [item for data in val_data_OG for item in data]
    Y_val = [item for data in val_label for item in data]
    
    # print(len(Y_train))
    # print(len(Y_val))
    
    test_data1 = [item for data in X_test_list1 for item in data]
    test_data_OG1 = [item for data in X_test_onlyGhost_list1 for item in data]
    test_label1 = [item for data in Y_test_list1 for item in data]
    MAE_data1 = [item for data in MAE_list_t1 for item in data]
    FINAL_QP_data1 = [item for data in FINAL_QP_list_t1 for item in data]
    
    test_data2 = [item for data in X_test_list2 for item in data]
    test_data_OG2 = [item for data in X_test_onlyGhost_list2 for item in data]
    test_label2 = [item for data in Y_test_list2 for item in data]
    MAE_data2 = [item for data in MAE_list_t2 for item in data]
    FINAL_QP_data2 = [item for data in FINAL_QP_list_t2 for item in data]
    
    test_data3 = [item for data in X_test_list3 for item in data]
    test_data_OG3 = [item for data in X_test_onlyGhost_list3 for item in data]
    test_label3 = [item for data in Y_test_list3 for item in data]
    MAE_data3 = [item for data in MAE_list_t3 for item in data]
    FINAL_QP_data3 = [item for data in FINAL_QP_list_t3 for item in data]
    
    print(len(MAE_data1))
    print(len(MAE_data2))
    print(len(MAE_data3))
    
                                                                                   
    best_threshold = 0
    best_accuracy = 0
    best_predicted_labels = []
    best_ground_truth_labels = []
    
    sameQP_best_threshold = 0
    sameQP_best_accuracy = 0
    sameQP_best_predicted_labels = []
    sameQP_best_ground_truth_labels = []
    
    largeQP_best_threshold = 0
    largeQP_best_accuracy = 0
    largeQP_best_predicted_labels = []
    largeQP_best_ground_truth_labels = []
            
    for threshold in np.arange(0.00, 1.01, 0.01):
        test_old = np.array([is_double_compressed(MAE_data1[i], FINAL_QP_data1[i], threshold) for i in range(600)])
        predicted_labels = test_old.astype(int)
        ground_truth_labels = np.array(test_label1)
        accuracy = np.sum(ground_truth_labels == predicted_labels) / len(ground_truth_labels)
    
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
            best_predicted_labels = predicted_labels
            best_ground_truth_labels = ground_truth_labels
            
    for threshold in np.arange(0.00, 1.01, 0.01):
        test_sameQP_old = np.array([is_double_compressed(MAE_data2[i], FINAL_QP_data2[i], threshold) for i in range(600)])
        same_predicted_labels = test_sameQP_old.astype(int)
        same_ground_truth_labels = np.array(test_label2)
        same_accuracy = np.sum(same_ground_truth_labels == same_predicted_labels) / len(same_ground_truth_labels)
    
        if same_accuracy > sameQP_best_accuracy:
            sameQP_best_accuracy = same_accuracy
            sameQP_best_threshold = threshold
            sameQP_best_predicted_labels = same_predicted_labels
            sameQP_best_ground_truth_labels = same_ground_truth_labels
                        
    for threshold in np.arange(0.00, 1.01, 0.01):
        test_largeQP_old = np.array([is_double_compressed(MAE_data3[i], FINAL_QP_data3[i], threshold) for i in range(600)])
        large_predicted_labels = test_largeQP_old.astype(int)
        large_ground_truth_labels = np.array(test_label3)
        large_accuracy = np.sum(large_ground_truth_labels == large_predicted_labels) / len(large_ground_truth_labels)
    
        if large_accuracy > largeQP_best_accuracy:
            largeQP_best_accuracy = large_accuracy
            largeQP_best_threshold = threshold
            largeQP_best_predicted_labels = large_predicted_labels
            largeQP_best_ground_truth_labels = large_ground_truth_labels       
            
            
    print(best_accuracy)
    print(sameQP_best_accuracy)
    print(largeQP_best_accuracy)
            
            
    best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = 0, None, None
    best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = 0, None, None
    
    best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = 0, None, None
    best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = 0, None, None
        
    for C_value in C_values['C']:    
        # SVMモデルのインスタンスを作成
        svm_model_RBF = SVC(kernel='rbf', C=C_value)
        svm_model_onlyGhost_RBF = SVC(kernel='rbf', C=C_value)
        
        svm_model_LINEAR = SVC(kernel='linear', C=C_value)
        svm_model_onlyGhost_LINEAR = SVC(kernel='linear', C=C_value)

        # 訓練データで訓練
        svm_model_RBF.fit(X_train, Y_train)
        svm_model_onlyGhost_RBF.fit(X_train_OG, Y_train)
        
        svm_model_LINEAR.fit(X_train, Y_train)
        svm_model_onlyGhost_LINEAR.fit(X_train_OG, Y_train)

        
        val_accuracy_RBF = accuracy_score(Y_val, svm_model_RBF.predict(X_val))
        val_accuracy_onlyGhost_RBF = accuracy_score(Y_val, svm_model_onlyGhost_RBF.predict(X_val_OG))
        
        val_accuracy_LINEAR = accuracy_score(Y_val, svm_model_LINEAR.predict(X_val))
        val_accuracy_onlyGhost_LINEAR = accuracy_score(Y_val, svm_model_onlyGhost_LINEAR.predict(X_val_OG))
        

        # 検証データでの精度が最も高かった場合、そのモデルを保存
        if val_accuracy_RBF > best_val_score_RBF:
            best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = val_accuracy_RBF, svm_model_RBF, C_value

        if val_accuracy_onlyGhost_RBF > best_val_score_onlyGhost_RBF:
            best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = val_accuracy_onlyGhost_RBF, svm_model_onlyGhost_RBF, C_value
            
        if val_accuracy_LINEAR > best_val_score_LINEAR:
            best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = val_accuracy_LINEAR, svm_model_LINEAR, C_value

        if val_accuracy_onlyGhost_LINEAR > best_val_score_onlyGhost_LINEAR:
            best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = val_accuracy_onlyGhost_LINEAR, svm_model_onlyGhost_LINEAR, C_value


    # テストデータで評価    
    predictions_RBF = best_svm_model_RBF.predict(test_data1)
    # predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test)
    accuracy_RBF = accuracy_score(test_label1, predictions_RBF)
    report_RBF = classification_report(test_label1, predictions_RBF, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label1, predictions_RBF)
    tnr_rbf_lqp1 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_rbf_lqp1 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'report_RBF:\n{report_RBF}')
    
    predictions_LINEAR = best_svm_model_LINEAR.predict(test_data1)
    # predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test)
    accuracy_LINEAR = accuracy_score(test_label1, predictions_LINEAR)
    report_LINEAR = classification_report(test_label1, predictions_LINEAR, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label1, predictions_LINEAR)
    tnr_linear_lqp1 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_linear_lqp1 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'report_LINEAR:\n{report_LINEAR}')
    
    same_predictions_RBF = best_svm_model_RBF.predict(test_data2)
    # same_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test_sameQP)
    same_accuracy_RBF = accuracy_score(test_label2, same_predictions_RBF)
    same_report_RBF = classification_report(test_label2, same_predictions_RBF, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label2, same_predictions_RBF)
    tnr_rbf_sqp = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_rbf_sqp = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'same_report_RBF:\n{same_report_RBF}')
    
    same_predictions_LINEAR = best_svm_model_LINEAR.predict(test_data2)
    # same_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test_sameQP)
    same_accuracy_LINEAR = accuracy_score(test_label2, same_predictions_LINEAR)
    same_report_LINEAR = classification_report(test_label2, same_predictions_LINEAR, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label2, same_predictions_LINEAR)
    tnr_linear_sqp = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_linear_sqp = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'same_report_LINEAR:\n{same_report_LINEAR}')
    
    large_predictions_RBF = best_svm_model_RBF.predict(test_data3)
    # large_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test_largeQP)
    large_accuracy_RBF = accuracy_score(test_label3, large_predictions_RBF)
    large_report_RBF = classification_report(test_label3, large_predictions_RBF, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label3, large_predictions_RBF)
    tnr_rbf_lqp2 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_rbf_lqp2 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'large_report_RBF:\n{large_report_RBF}')
    
    large_predictions_LINEAR = best_svm_model_LINEAR.predict(test_data3)
    # large_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test_largeQP)
    large_accuracy_LINEAR = accuracy_score(test_label3, large_predictions_LINEAR)
    large_report_LINEAR = classification_report(test_label3, large_predictions_LINEAR, digits=4, zero_division=1)
    conf_matrix = confusion_matrix(test_label3, large_predictions_LINEAR)
    tnr_linear_lqp2 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_linear_lqp2 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'large_report_LINEAR:\n{large_report_LINEAR}')
    
    
    
    
    # テストデータで評価    
    predictions_RBF2 = best_svm_model_onlyGhost_RBF.predict(test_data_OG1)
    # predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test)
    accuracy_RBF2 = accuracy_score(test_label1, predictions_RBF2)
    report_RBF2 = classification_report(test_label1, predictions_RBF2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label1, predictions_RBF2)
    tnr_rbf2_lqp1 = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_rbf2_lqp1 = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'report_RBF2:\n{report_RBF2}')
    
    predictions_LINEAR2 = best_svm_model_onlyGhost_LINEAR.predict(test_data_OG1)
    # predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test)
    accuracy_LINEAR2 = accuracy_score(test_label1, predictions_LINEAR2)
    report_LINEAR2 = classification_report(test_label1, predictions_LINEAR2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label1, predictions_LINEAR2)
    tnr_linear2_lqp1 = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_linear2_lqp1 = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'report_LINEAR2:\n{report_LINEAR2}')
    
    same_predictions_RBF2 = best_svm_model_onlyGhost_RBF.predict(test_data_OG2)
    # same_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test_sameQP)
    same_accuracy_RBF2 = accuracy_score(test_label2, same_predictions_RBF2)
    same_report_RBF2 = classification_report(test_label2, same_predictions_RBF2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label2, same_predictions_RBF2)
    tnr_rbf2_sqp = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_rbf2_sqp = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'same_report_RBF2:\n{same_report_RBF2}')
    
    same_predictions_LINEAR2 = best_svm_model_onlyGhost_LINEAR.predict(test_data_OG2)
    # same_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test_sameQP)
    same_accuracy_LINEAR2 = accuracy_score(test_label2, same_predictions_LINEAR2)
    same_report_LINEAR2 = classification_report(test_label2, same_predictions_LINEAR2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label2, same_predictions_LINEAR2)
    tnr_linear2_sqp = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_linear2_sqp = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'same_report_LINEAR2:\n{same_report_LINEAR2}')
    
    large_predictions_RBF2 = best_svm_model_onlyGhost_RBF.predict(test_data_OG3)
    # large_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test_largeQP)
    large_accuracy_RBF2 = accuracy_score(test_label3, large_predictions_RBF2)
    large_report_RBF2 = classification_report(test_label3, large_predictions_RBF2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label3, large_predictions_RBF2)
    tnr_rbf2_lqp2 = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_rbf2_lqp2 = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'large_report_RBF2:\n{large_report_RBF2}')
    
    large_predictions_LINEAR2 = best_svm_model_onlyGhost_LINEAR.predict(test_data_OG3)
    # large_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test_largeQP)
    large_accuracy_LINEAR2 = accuracy_score(test_label3, large_predictions_LINEAR2)
    large_report_LINEAR2 = classification_report(test_label3, large_predictions_LINEAR2, digits=4, zero_division=1)
    conf_matrix2 = confusion_matrix(test_label3, large_predictions_LINEAR2)
    tnr_linear2_lqp2 = conf_matrix2[0, 0] / (conf_matrix2[0, 0] + conf_matrix2[0, 1])
    tpr_linear2_lqp2 = conf_matrix2[1, 1] / (conf_matrix2[1, 0] + conf_matrix2[1, 1])
    print(f'large_report_LINEAR2:\n{large_report_LINEAR2}')
    
    
    
    # テストデータで評価
    test_old = classification_report(best_ground_truth_labels, best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0, digits=4)
    conf_matrix = confusion_matrix(best_ground_truth_labels, best_predicted_labels)
    tnr_old_lqp1 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_old_lqp1 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary old_model:\n{test_old}')
    
    test_sameQP_old = classification_report(sameQP_best_ground_truth_labels, sameQP_best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0, digits=4)
    conf_matrix = confusion_matrix(sameQP_best_ground_truth_labels, sameQP_best_predicted_labels)
    tnr_old_sqp = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_old_sqp = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary old_model:\n{test_sameQP_old}')
    
    test_largeQP_old = classification_report(largeQP_best_ground_truth_labels, largeQP_best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0, digits=4)
    conf_matrix = confusion_matrix(largeQP_best_ground_truth_labels, largeQP_best_predicted_labels)
    tnr_old_lqp2 = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_old_lqp2 = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary old_model:\n{test_largeQP_old}')
    
    
    

    # Test結果を保存
    
    result_row ={'C_RBF_LQP1':best_c_value_RBF,'Score_RBF_LQP1': accuracy_RBF, 'tnr_rbf_lqp1':tnr_rbf_lqp1, 'tpr_rbf_lqp1':tpr_rbf_lqp1,
                'C_RBF_SQP': best_c_value_RBF, 'Score_RBF_SQP': same_accuracy_RBF, 'tnr_rbf_sqp':tnr_rbf_sqp, 'tpr_rbf_sqp':tpr_rbf_sqp,
                'C_RBF_LQP2': best_c_value_RBF,'Score_RBF_LQP2': large_accuracy_RBF, 'tnr_rbf_lqp2':tnr_rbf_lqp2, 'tpr_rbf_lqp2':tpr_rbf_lqp2,
                                    
                'C_LINEAR_LQP1': best_c_value_LINEAR,'Score_LINEAR_LQP1':accuracy_LINEAR, 'tnr_linear_lqp1':tnr_linear_lqp1, 'tpr_linear_lqp1':tpr_linear_lqp1,
                'C_LINEAR_SQP': best_c_value_LINEAR,'Score_LINEAR_SQP':same_accuracy_LINEAR, 'tnr_linear_sqp':tnr_linear_sqp, 'tpr_linear_sqp':tpr_linear_sqp,
                'C_LINEAR_LQP2': best_c_value_LINEAR,'Score_LINEAR_LQP2':large_accuracy_LINEAR, 'tnr_linear_lqp2':tnr_linear_lqp2, 'tpr_linear_lqp2':tpr_linear_lqp2,
                 
                'C_RBF2_LQP1':best_c_value_onlyGhost_RBF,'Score_RBF2_LQP1': accuracy_RBF2, 'tnr_rbf2_lqp1':tnr_rbf2_lqp1, 'tpr_rbf2_lqp1':tpr_rbf2_lqp1,
                'C_RBF2_SQP': best_c_value_onlyGhost_RBF, 'Score_RBF2_SQP': same_accuracy_RBF2, 'tnr_rbf2_sqp':tnr_rbf2_sqp, 'tpr_rbf2_sqp':tpr_rbf2_sqp,
                'C_RBF2_LQP2': best_c_value_onlyGhost_RBF,'Score_RBF2_LQP2': large_accuracy_RBF2, 'tnr_rbf2_lqp2':tnr_rbf2_lqp2, 'tpr_rbf2_lqp2':tpr_rbf2_lqp2,
                                    
                'C_LINEAR2_LQP1': best_c_value_onlyGhost_LINEAR,'Score_LINEAR2_LQP1':accuracy_LINEAR2, 'tnr_linear2_lqp1':tnr_linear2_lqp1, 'tpr_linear2_lqp1':tpr_linear2_lqp1,
                'C_LINEAR2_SQP': best_c_value_onlyGhost_LINEAR,'Score_LINEAR2_SQP':same_accuracy_LINEAR2, 'tnr_linear2_sqp':tnr_linear2_sqp, 'tpr_linear2_sqp':tpr_linear2_sqp,
                'C_LINEAR2_LQP2': best_c_value_onlyGhost_LINEAR,'Score_LINEAR2_LQP2':large_accuracy_LINEAR2, 'tnr_linear2_lqp2':tnr_linear2_lqp2, 'tpr_linear2_lqp2':tpr_linear2_lqp2,
                                                        
                'Threshold_LQP1':best_threshold, 'LQP1_old':best_accuracy, 'tnr_old_lqp1':tnr_old_lqp1, 'tpr_old_lqp1':tpr_old_lqp1,
                'Threshold_SQP':sameQP_best_threshold, 'SQP_old':sameQP_best_accuracy, 'tnr_old_sqp':tnr_old_sqp, 'tpr_old_sqp':tpr_old_sqp,
                'Threshold_LQP2':largeQP_best_threshold, 'LQP2_old':largeQP_best_accuracy, 'tnr_old_lqp2':tnr_old_lqp2, 'tpr_old_lqp2':tpr_old_lqp2}


    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)

    

<Fold-1>
Train indices: [0 1 2 3 4 5 6 8]
Test indices: [7]
600
600
600
0.8416666666666667
0.5016666666666667
0.5566666666666666
report_RBF:
              precision    recall  f1-score   support

           0     0.9544    0.9067    0.9299       300
           1     0.9111    0.9567    0.9333       300

    accuracy                         0.9317       600
   macro avg     0.9327    0.9317    0.9316       600
weighted avg     0.9327    0.9317    0.9316       600

report_LINEAR:
              precision    recall  f1-score   support

           0     0.9654    0.8367    0.8964       300
           1     0.8559    0.9700    0.9094       300

    accuracy                         0.9033       600
   macro avg     0.9106    0.9033    0.9029       600
weighted avg     0.9106    0.9033    0.9029       600

same_report_RBF:
              precision    recall  f1-score   support

           0     0.8168    0.9067    0.8594       300
           1     0.8951    0.7967    0.8430       300

    accur

In [16]:
# 各統計情報を100倍して小数点第4位までの表記に変更
statistics_data = {
    'Model': ['RBF_LQP1', 'RBF_SQP', 'RBF_LQP2', 'LINEAR_LQP1', 'LINEAR_SQP', 'LINEAR_LQP2', 'OLD_LQP1', 'OLD_SQP', 'OLD_LQP2'],
    'Average TNR': [round(results['tnr_rbf_lqp1'].mean() * 100, 2), round(results['tnr_rbf_sqp'].mean() * 100, 2), round(results['tnr_rbf_lqp2'].mean() * 100, 2), round(results['tnr_linear_lqp1'].mean() * 100, 2), round(results['tnr_linear_sqp'].mean() * 100, 2), round(results['tnr_linear_lqp2'].mean() * 100, 2),round(results['tnr_old_lqp1'].mean() * 100, 2), round(results['tnr_old_sqp'].mean() * 100, 2), round(results['tnr_old_lqp2'].mean() * 100, 2)],
    'Average TPR': [round(results['tpr_rbf_lqp1'].mean() * 100, 2), round(results['tpr_rbf_sqp'].mean() * 100, 2), round(results['tpr_rbf_lqp2'].mean() * 100, 2), round(results['tpr_linear_lqp1'].mean() * 100, 2), round(results['tpr_linear_sqp'].mean() * 100, 2), round(results['tpr_linear_lqp2'].mean() * 100, 2),round(results['tpr_old_lqp1'].mean() * 100, 2), round(results['tpr_old_sqp'].mean() * 100, 2), round(results['tpr_old_lqp2'].mean() * 100, 2)],
    'Average Test Score': [round(results['Score_RBF_LQP1'].mean() * 100, 2), round(results['Score_RBF_SQP'].mean() * 100, 2), round(results['Score_RBF_LQP2'].mean() * 100, 2), round(results['Score_LINEAR_LQP1'].mean() * 100, 2), round(results['Score_LINEAR_SQP'].mean() * 100, 2), round(results['Score_LINEAR_LQP2'].mean() * 100, 2),round(results['LQP1_old'].mean() * 100, 2), round(results['SQP_old'].mean() * 100, 2), round(results['LQP2_old'].mean() * 100, 2)],
    'Standard Deviation': [round(results['Score_RBF_LQP1'].std() * 100, 2), round(results['Score_RBF_SQP'].std() * 100, 2), round(results['Score_RBF_LQP2'].std() * 100, 2), round(results['Score_LINEAR_LQP1'].std() * 100, 2), round(results['Score_LINEAR_SQP'].std() * 100, 2), round(results['Score_LINEAR_LQP2'].std() * 100, 2),round(results['LQP1_old'].std() * 100, 2), round(results['SQP_old'].std() * 100, 2), round(results['LQP2_old'].std() * 100, 2)],
    'Max Test Score': [round(results['Score_RBF_LQP1'].max() * 100, 2), round(results['Score_RBF_SQP'].max() * 100, 2), round(results['Score_RBF_LQP2'].max() * 100, 2), round(results['Score_LINEAR_LQP1'].max() * 100, 2), round(results['Score_LINEAR_SQP'].max() * 100, 2), round(results['Score_LINEAR_LQP2'].max() * 100, 2),round(results['LQP1_old'].max() * 100, 2), round(results['SQP_old'].max() * 100, 2), round(results['LQP2_old'].max() * 100, 2)],
    'Min Test Score': [round(results['Score_RBF_LQP1'].min() * 100, 2), round(results['Score_RBF_SQP'].min() * 100, 2), round(results['Score_RBF_LQP2'].min() * 100, 2), round(results['Score_LINEAR_LQP1'].min() * 100, 2), round(results['Score_LINEAR_SQP'].min() * 100, 2), round(results['Score_LINEAR_LQP2'].min() * 100, 2),round(results['LQP1_old'].min() * 100, 2), round(results['SQP_old'].min() * 100, 2), round(results['LQP2_old'].min() * 100, 2)],
}

# DataFrameを作成
statistics_df = pd.DataFrame(statistics_data)

# 表示
print(statistics_df)

         Model  Average TNR  Average TPR  Average Test Score  Standard Deviation  Max Test Score  Min Test Score
0     RBF_LQP1        89.44        95.56               92.50                0.99           93.67           90.67
1      RBF_SQP        89.44        83.44               86.44                2.09           90.17           84.50
2     RBF_LQP2        89.44        41.33               65.39                3.22           69.33           62.17
3  LINEAR_LQP1        83.22        96.67               89.94                0.30           90.33           89.50
4   LINEAR_SQP        83.22        87.96               85.59                1.28           87.33           83.33
5  LINEAR_LQP2        83.22        50.44               66.83                1.26           68.33           64.67
6     OLD_LQP1        81.33        87.00               84.17                0.00           84.17           84.17
7      OLD_SQP        99.00         1.33               50.17                0.00           50.17

In [17]:
# 各統計情報を100倍して小数点第4位までの表記に変更
statistics_data2 = {
    'Model': ['RBF_LQP1', 'RBF_SQP', 'RBF_LQP2', 'LINEAR_LQP1', 'LINEAR_SQP', 'LINEAR_LQP2', 'OLD_LQP1', 'OLD_SQP', 'OLD_LQP2'],
    'Average TNR': [round(results['tnr_rbf2_lqp1'].mean() * 100, 2), round(results['tnr_rbf2_sqp'].mean() * 100, 2), round(results['tnr_rbf2_lqp2'].mean() * 100, 2), round(results['tnr_linear2_lqp1'].mean() * 100, 2), round(results['tnr_linear2_sqp'].mean() * 100, 2), round(results['tnr_linear2_lqp2'].mean() * 100, 2),round(results['tnr_old_lqp1'].mean() * 100, 2), round(results['tnr_old_sqp'].mean() * 100, 2), round(results['tnr_old_lqp2'].mean() * 100, 2)],
    'Average TPR': [round(results['tpr_rbf2_lqp1'].mean() * 100, 2), round(results['tpr_rbf2_sqp'].mean() * 100, 2), round(results['tpr_rbf2_lqp2'].mean() * 100, 2), round(results['tpr_linear2_lqp1'].mean() * 100, 2), round(results['tpr_linear2_sqp'].mean() * 100, 2), round(results['tpr_linear2_lqp2'].mean() * 100, 2),round(results['tpr_old_lqp1'].mean() * 100, 2), round(results['tpr_old_sqp'].mean() * 100, 2), round(results['tpr_old_lqp2'].mean() * 100, 2)],
    'Average Test Score': [round(results['Score_RBF2_LQP1'].mean() * 100, 2), round(results['Score_RBF2_SQP'].mean() * 100, 2), round(results['Score_RBF2_LQP2'].mean() * 100, 2), round(results['Score_LINEAR2_LQP1'].mean() * 100, 2), round(results['Score_LINEAR2_SQP'].mean() * 100, 2), round(results['Score_LINEAR2_LQP2'].mean() * 100, 2),round(results['LQP1_old'].mean() * 100, 2), round(results['SQP_old'].mean() * 100, 2), round(results['LQP2_old'].mean() * 100, 2)],
    'Standard Deviation': [round(results['Score_RBF2_LQP1'].std() * 100, 2), round(results['Score_RBF2_SQP'].std() * 100, 2), round(results['Score_RBF2_LQP2'].std() * 100, 2), round(results['Score_LINEAR2_LQP1'].std() * 100, 2), round(results['Score_LINEAR2_SQP'].std() * 100, 2), round(results['Score_LINEAR2_LQP2'].std() * 100, 2),round(results['LQP1_old'].std() * 100, 2), round(results['SQP_old'].std() * 100, 2), round(results['LQP2_old'].std() * 100, 2)],
    'Max Test Score': [round(results['Score_RBF2_LQP1'].max() * 100, 2), round(results['Score_RBF2_SQP'].max() * 100, 2), round(results['Score_RBF2_LQP2'].max() * 100, 2), round(results['Score_LINEAR2_LQP1'].max() * 100, 2), round(results['Score_LINEAR2_SQP'].max() * 100, 2), round(results['Score_LINEAR2_LQP2'].max() * 100, 2),round(results['LQP1_old'].max() * 100, 2), round(results['SQP_old'].max() * 100, 2), round(results['LQP2_old'].max() * 100, 2)],
    'Min Test Score': [round(results['Score_RBF2_LQP1'].min() * 100, 2), round(results['Score_RBF2_SQP'].min() * 100, 2), round(results['Score_RBF2_LQP2'].min() * 100, 2), round(results['Score_LINEAR2_LQP1'].min() * 100, 2), round(results['Score_LINEAR2_SQP'].min() * 100, 2), round(results['Score_LINEAR2_LQP2'].min() * 100, 2),round(results['LQP1_old'].min() * 100, 2), round(results['SQP_old'].min() * 100, 2), round(results['LQP2_old'].min() * 100, 2)],
}

# DataFrameを作成
statistics_df2 = pd.DataFrame(statistics_data2)

# 表示
print(statistics_df2)

         Model  Average TNR  Average TPR  Average Test Score  Standard Deviation  Max Test Score  Min Test Score
0     RBF_LQP1        83.63        93.63               88.63                0.39           89.33           88.17
1      RBF_SQP        83.63        88.07               85.85                0.43           86.50           85.33
2     RBF_LQP2        83.63        21.70               52.67                1.86           55.67           51.00
3  LINEAR_LQP1        78.67        86.96               82.81                2.84           90.17           81.17
4   LINEAR_SQP        78.67        43.56               61.11                2.80           63.33           53.83
5  LINEAR_LQP2        78.67        34.41               56.54                2.96           57.83           48.67
6     OLD_LQP1        81.33        87.00               84.17                0.00           84.17           84.17
7      OLD_SQP        99.00         1.33               50.17                0.00           50.17

In [18]:
print(results['C_RBF_LQP1'])
print(results['C_LINEAR_LQP1'])
print()
print(results['C_RBF2_LQP1'])
print(results['C_LINEAR2_LQP1'])

0      10
1     100
2      10
3      10
4      10
5    1000
6      10
7    1000
8     100
Name: C_RBF_LQP1, dtype: object
0     100
1     100
2    1000
3     100
4      10
5    1000
6     100
7      10
8     100
Name: C_LINEAR_LQP1, dtype: object

0    2000
1    1000
2    4000
3    2000
4      10
5    4000
6     100
7    2000
8     100
Name: C_RBF2_LQP1, dtype: object
0      1
1    100
2      1
3      1
4      1
5    0.1
6      1
7      1
8      1
Name: C_LINEAR2_LQP1, dtype: object


In [19]:
statistics_df.to_csv('statistics_data3.csv', index=False)
statistics_df2.to_csv('statistics2_data3.csv', index=False)